In [26]:
import os
import numpy as np
import quandl
from datetime import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.offline as py
import collections
py.init_notebook_mode(connected=True)
import plotly.figure_factory as ff
from datetime import datetime
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import json 
import pandas as pd
from sklearn.metrics import confusion_matrix

consumer_key = 'sCEbnH8oUTQ6x2X4NdcOnQQ9Q'
consumer_secret = 'Fm67NK9ATwWKgA1uIE9lDrTIGCSCim7nAZZAly19r6Qeo08tN9'
access_token = '540629516-dYg6Z5Ze4J2etUxoiLLs4WVbgN2QEtdq1oZlNFL7'
access_token_secret = 'skHdmjkgE5lGrVpLUJn1JPTKYranINpU1MArbKKMPiuL1'

quandl.ApiConfig.api_key = 'N6auK8fALzKo_6npF-44'

In [2]:
#Function to Read quandl data
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [3]:
#Calling function to read Kraken exchange's bitcoin  data
def get_BitCoin_TestData(initialDate, endDate):
    return quandl.get("BCHARTS/KRAKENUSD", start_date=initialDate, end_date=endDate)

In [4]:
#Function to label data as positive, negative and neutral by difference of closing and opening price above and below 200
#Also performs standardizing difference field from -1 t0 1 and store into CSV file
def get_BitcoinDataIntoCSV(testBitCoinData,filename):
    lstTkn=[]
    diff=[]
    for gen in range((len(testBitCoinData))):
        change = testBitCoinData["Close"][gen]-testBitCoinData["Open"][gen]
        diff.append(change)
        if testBitCoinData["Close"][gen]-testBitCoinData["Open"][gen] > 0:
            if change > 200:
                lstTkn.append("Positive")
            else:
                lstTkn.append("Neutral")
        else:
            if change < -200:
                lstTkn.append("Negative")
            else:
                lstTkn.append("Neutral")

    testBitCoinData["Change"]=lstTkn
    testBitCoinData["Diff"]=diff
    minval=min(testBitCoinData["Diff"])
    maxval=max(testBitCoinData["Diff"])
    sub=maxval-minval

    new=[]
    for gen in range((len(testBitCoinData))):
        n = 2*((testBitCoinData["Diff"][gen]-minval)/sub)-1
        new.append(n)


    testBitCoinData["Score"]=new
    testBitCoinData.to_csv(filename)

In [5]:
#Sets twitter authentication
def set_Tweepy():
    auth = OAuthHandler(consumer_key, consumer_secret)
    # set access token and secret
    auth.set_access_token(access_token, access_token_secret)
    #create tweepy API object to fetch tweets
    api = tweepy.API(auth)

In [6]:
#To clean tweet with special character and links
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("https?:.*(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", tweet).split())

In [7]:
#Retrieve the polarity score of individual tweet
def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(clean_tweet(tweet))
        # set sentiment
        return analysis.sentiment.polarity 

In [8]:
#Function to make CSV file from JSON file's data with required set of columns
def makeExcelNew(tweets,timeStamp,inputFile):
    outFile=inputFile.split('.')[0];
    outputFile=outFile+'.csv'
    tweetArr=[]
    tweetPolarityArr=[]
    timeStampArr=[]
    for tweet in tweets:
        tweetArr.append(clean_tweet(tweet))
        tweetPolarityArr.append(get_tweet_sentiment(tweet))
    for time in timeStamp:
        t=time[0:10]
        timeStampArr.append(t)
    df = pd.DataFrame(columns=['Polarity','Time', 'Tweets'])
    for i in range(len(tweetArr)):
        df = df.append({'Polarity':tweetPolarityArr[i],'Time':timeStampArr[i],'Tweets':tweetArr[i]},ignore_index=True)
    df.to_csv(outputFile, sep=',', encoding='utf-8')
    return 0

In [9]:
#Function to read JSON file
def get_tweets(inputJson):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
        retweets=[]
        timeStamp=[]
        inputFile=inputJson
        try:
            with open(inputFile, 'r') as f:                
                for line in f:
                    if len(line) > 1:                                
                        tweetLine = json.loads(line)      
                        for tweetText in tweetLine:
                            if int(tweetText['retweets']) > 0:
                                if tweetText['text'] not in tweets:
                                    tweets.append(tweetText['text'])
                                    timeStamp.append(tweetText['timestamp'])
                                else:
                                    retweets.append(tweetText['text'])                                                      
                            else:
                                tweets.append(tweetText['text'])
                                timeStamp.append(tweetText['timestamp'])
                    else:
                        break
                makeExcelNew(tweets,timeStamp,inputFile)
                return tweets
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [10]:
#To Merge (average the polarity score of each tweet on daily basis) all month's twitter data into file 
#So one CSV file as per training and testing
def MergeALLCSV(outputfilename):
    lstPol=[]
    lstDat=[]
    initialDate = datetime.strptime("2017-06-01","%Y-%m-%d")
    if outputfilename == 'TwitterTest.csv':
        filelist=['tweetsJan.csv','tweetsFeb.csv']
    else:
        filelist=['tweetsJun.csv','tweetsJul.csv', 'tweetsAug.csv','tweetsSep.csv', 'tweetsOct.csv','tweetsNov.csv','tweetsDec.csv']
            
    Count=collections.Counter()
    d={}
    for fileitem in filelist:
        #print(fileitem)
        data=pd.read_csv(fileitem)
        data.dropna()
        
        #d={}
        Count.update(data['Time'])
        for gen in range((len(data))):
            datetime_object = datetime.strptime(data['Time'][gen],"%Y-%m-%d")
            if datetime_object != None and datetime_object >= initialDate:
                if(data['Time'][gen] in d):
                    #print("Match")
                    d[data['Time'][gen]] =  d[data['Time'][gen]] + data['Polarity'][gen]
                else:
                    d[data['Time'][gen]]=data['Polarity'][gen]
        
    for key, value in d.items():
        lstDat.append(key)
        lstPol.append(value/Count[key])
            
    df = pd.DataFrame(columns=['Date','Score'])
    for i in range(len(lstPol)):
        df = df.append({'Date':lstDat[i],'Score':lstPol[i]},ignore_index=True)
    df.to_csv(outputfilename, sep=',', encoding='utf-8')

In [11]:
def TrainModelofTwitter(tweetDataFile,BitCoinDataFile):
    trainTweet = pd.read_csv(tweetDataFile)
    trainBitCoin = pd.read_csv(BitCoinDataFile)
    lstChange=[]
    #Below values are obtained by running multiple iterations and comparing the output in terms of confusion matrix
    #for least error and higher accuracy
    for i in range(len(trainTweet)):
        if trainTweet["Score"][i] >= 0.13:  
            lstChange.append("Positive")
        elif trainTweet["Score"][i] <= -0.05:
            lstChange.append("Negative")
        else:
            lstChange.append("Neutral")
            
    trainTweet["Change"]=lstChange
    trainTweet.to_csv(tweetDataFile)

In [12]:
#Fucntion to check if both data files are compatible by checking the twitter's data of particular date with Bitcoin's data
#If twitter data for a date is not found then we dropped that date's Bitcoin data 
def makeBothDataCompatible(tweetDataFile, BitCoinDataFile,outputBitcoinDataFile):
    lstDate=[]
    lstVal=[]
    lstScore=[]
    train = pd.read_csv(BitCoinDataFile)
    test = pd.read_csv(tweetDataFile)
    test['Date'] = pd.to_datetime(test['Date']).dt.date
    train['Date'] = pd.to_datetime(test['Date']).dt.date
    for i in range(len(test['Date'])):
        for j in range(len(train['Date'])):
            if(test['Date'][i]==train['Date'][j]):
                lstDate.append(train['Date'][j])
                lstScore.append(train['Score'][j])
                lstVal.append(train['Change'][j])

    df = pd.DataFrame(columns=['Date','Score','Change'])
    for i in range(len(lstDate)):
        df = df.append({'Date':lstDate[i],'Score':lstScore[i],'Change':lstVal[i]},ignore_index=True)  
    df.to_csv(outputBitcoinDataFile, sep=',', encoding='utf-8')    

In [13]:
#Fucntion to generate confusion matrix
def EvaluateModel(twitterFile, bitcoinFile):
    train = pd.read_csv(twitterFile)
    test = pd.read_csv(bitcoinFile)
    train = train.dropna()
    test = test.dropna()
    return confusion_matrix(train['Change'], test['Change'], labels=["Positive", "Neutral", "Negative"])

In [14]:
def plotVariations(twitterFile, bitcoinFile):
    twt=pd.read_csv(twitterFile)
    btc=pd.read_csv(bitcoinFile)

    lsttmp =[]
    for i in range(len(btc['Date'])):
        lsttmp.append(datetime.strptime((btc['Date'][i]), "%Y-%m-%d"))
    btc['new']=lsttmp
    btc = btc.sort_values(by=['new'])


    lsttmp1 =[]
    for i in range(len(twt['Date'])):
        lsttmp1.append(datetime.strptime((twt['Date'][i]), "%Y-%m-%d"))
    twt['new']=lsttmp1
    twt = twt.sort_values(by=['new'])

    init_notebook_mode(connected=True)
    trace = go.Scatter(x=btc["Date"], y=btc["Score"], mode='lines+markers')
    trace1 = go.Scatter(x=twt["Date"], y=twt["Score"], mode='lines+markers')
    py.iplot([trace])
    py.iplot([trace1])

In [15]:
#Getting the training data of Bitcoin and storing into CSV
data = get_BitCoin_TestData("2017-06-01", "2017-12-31")
get_BitcoinDataIntoCSV(data,"BitCoinJun-Dec.csv")

In [16]:
#Getting the training data of Twitter and storing into CSV
set_Tweepy()
jsonList=['tweetsJun.json','tweetsJul.json','tweetsAug.json','tweetsSep.json','tweetsOct.json','tweetsNov.json','tweetsDec.json']
for jsonFile in jsonList:
    get_tweets(jsonFile)

In [17]:
#Merging all CSV
MergeALLCSV("TwitterTrain.csv")

In [18]:
#Classification of Twitter tweets
TrainModelofTwitter("TwitterTrain.csv","BitCoinJun-Dec.csv")
makeBothDataCompatible("TwitterTrain.csv","BitCoinJun-Dec.csv","BitCoinTrain.csv")

In [19]:
#Evaluate the Training Model
EvaluateModel("TwitterTrain.csv","BitCoinTrain.csv")
#Columns are Bitcoin labled data as positive, neutral and negative (Actual)
#Rows are  Twitter tweets as positive, neutral and negative (Predicted)

array([[ 1, 34,  2],
       [13, 85,  7],
       [ 2,  2,  1]], dtype=int64)

In [20]:
plotVariations("TwitterTrain.csv","BitCoinTrain.csv")

In [21]:
#Getting the test data of Bitcoin and storing into CSV
data = get_BitCoin_TestData("2018-01-01", "2018-02-28")
get_BitcoinDataIntoCSV(data,"BitCoinJan-Feb.csv")

In [22]:
#Getting the test data of Twitter and storing into CSV
jsonList=['tweetsJan.json','tweetsFeb.json']
for jsonFile in jsonList:
    get_tweets(jsonFile)
MergeALLCSV("TwitterTest.csv")

In [23]:
#Classification of Twitter tweets for testing data
TrainModelofTwitter("TwitterTest.csv","BitCoinJan-Feb.csv")
makeBothDataCompatible("TwitterTest.csv","BitCoinJan-Feb.csv","BitCoinTest.csv")

In [24]:
#Evaluate the test model
EvaluateModel("TwitterTest.csv","BitCoinTest.csv")
#Columns are Bitcoin labled data as positive, neutral and negative (Actual)
#Rows are  Twitter tweets as positive, neutral and negative (Predicted)

array([[ 4,  3,  8],
       [10,  7,  9],
       [ 0,  0,  0]], dtype=int64)

In [25]:
plotVariations("TwitterTest.csv","BitCoinTest.csv")